# Getting Started with ESA++

ESA++ wraps PowerWorld's SimAuto COM interface into Pythonic indexing.
This notebook walks through opening a case, reading and writing data,
and running a power flow.

## Opening a Case

Create a `GridWorkBench` and import the component classes you need.

```python
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch

wb = GridWorkBench("path/to/case.pwb")
```

In [ ]:
# This cell is hidden in the documentation.
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch
import numpy as np
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

## Reading Data

Use `wb[ComponentType, fields]` to query data. A single field returns a
two-column DataFrame (keys + values); a list of fields returns all
requested columns with keys prepended automatically.

In [ ]:
wb[Bus, "BusPUVolt"].head()

In [ ]:
wb[Gen, ["GenMW", "GenMVR", "GenStatus"]].head()

The result is a regular pandas DataFrame, so standard filtering works:

In [ ]:
gens = wb[Gen, ["GenMW", "GenMVR", "GenStatus"]]
online_gens = gens[gens["GenStatus"] == "Closed"]
online_gens.head()

In [ ]:
loads = wb[Load, ["LoadMW", "LoadMVR"]]
print(f"Total system load: {loads['LoadMW'].sum():.1f} MW")

## Writing Data

Assign with the same syntax. A scalar broadcasts to every row.

In [ ]:
wb[Gen, "GenMW"] = 100.0
wb[Gen, "GenMW"].head()

## Power Flow

`pflow()` solves the AC power flow and returns complex bus voltages.

In [ ]:
V = wb.pflow()
V.head()

In [ ]:
vmag = np.abs(V)
print(f"Voltage range: {vmag.min():.4f} – {vmag.max():.4f} pu")
print(f"Buses below 0.98 pu: {(vmag < 0.98).sum()}")

## Branch Data

Query branch loading the same way. The result is a plain DataFrame
you can sort, filter, or export as needed.

In [ ]:
branches = wb[Branch, ["BusNum", "BusNum:1", "LineMVA", "LinePercent"]]
branches.sort_values("LinePercent", ascending=False).head()